# Assignment 6

### 1.复习上课内容

### 2. 回答一下理论题目

#### 1. What is independent assumption in Naive bayes ?

<font color = black>朴素贝叶斯的独立假设是指给定类标签，样本的各个特征之间是相互独立的，即
    $P(X = x|Y = y)=\prod_{i=0}^nP(x_i|Y=y)$  
    其中$x_i$是$x$的一个特征分量。
</font>


#### 2. What is MAP(maximum a posterior) and ML(maximum likelihood) ?

对于参数估计方法有两类：最大后验概率估计和极大似然估计。

MAP假设模型参数是不确定的，参数源自一个潜在的分布，希望从数据中推知该分布，具体做法是不断调整模型参数使得模型能够产生该数据样本的概率最大。

MLE假设模型参数是确定的，可以直接通过最大化数据样本的出现可能性来求出这个参数，没有先验估计，其结果完全依赖于观察的数据。


#### 3. What is support vector in SVM?


支持向量是指样本中距离超平面最近的一些数据点，分类的超平面是靠这些数据点来确定的。

SVM思想：一些线性不可分的问题可能是非线性可分的，也就是在高维空间中存在分离超平面（separating hyperplane）
使用非线性函数从原始的特征空间映射至更高维的空间，转化为线性可分问题



#### 4. What is the intuition behind SVM ?

SVM背后的直觉假设是数据是线性可分的（或可以通过非线性函数映射到线性可分的空间），可以通过一个超平面对数据空间进行切分。

#### 5. Shortly describ what 'random' means in random forest ?

随机森林是非常具有代表性的Bagging集成算法，它的所有基评估器都是决策树，分类树组成的森林就叫做随机森
林分类器，回归树所集成的森林就叫做随机森林回归器。

每一棵决策树的训练数据集是随机抽取的，而且特征也是随机抽取的。

#### 6. What cariterion does XGBoost use to find the best split point in a tree ?

XGBoost是串行生成CART树，但是XGBoost在处理特征时可以做到并行处理。在最优切分点的选择上，XGBoost首先根据每个特征大小对训练数据进行排序，保存为block结构，block个数与特征数量相等。然后对每个block结构选择最佳的特征切分点，节点切分标准是选择目标函数下降最大的点作为每个block结构的最佳切分点。最后比较每个block结构的最佳特征切分点的目标函数下降的增益，下降最大的切分点为最终的最佳切分点。

参考
https://baijiahao.baidu.com/s?id=1621144293288522535&wfr=spider&for=pc

https://baijiahao.baidu.com/s?id=1621144293288522535&wfr=spider&for=pc%EF%BC%89


### 3. Practial part

##### Problem description: In this part you are going to build a classifier to detect if a piece of news is published by the Xinhua news agency (新华社）.

#### Hints:

###### 1. Firstly, you have to come up with a way to represent the news. (Vectorize the sentence, you can find different ways to do so online)  

###### 2. Secondly,  pick a machine learning algorithm that you think is suitable for this task

In [7]:
import pandas as pd
import numpy as np
import re
import jieba
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.utils import shuffle
from sklearn.metrics import classification_report
from tqdm import tqdm_notebook

In [14]:
data = pd.read_csv('../data/sqlResult_1558435.csv',encoding = 'gb18030')
data = data[['source','content']]
data = data.dropna()
data = shuffle(data)
data.head()

,source,content
38210,新华社,新华社照片，外代，2017年4月23日\n（外代二线）篮球——NBA季后赛：猛龙胜雄鹿\n4...
81321,新华社,新华社照片，南昌，2017年5月31日\n（体育）（2）“体操娃娃”的体育梦\n倒立、压腿、...
22248,新华社,新华社北京４月９日电 ２０１７年男子冰球世锦赛乙级Ｂ组９日在新西兰展开了第四轮的争夺，此...
81613,新华社,新华社郑州５月３１日电（记者李丽静）记者３１日从河南省濮阳市华龙区人民法院获悉，该院２０...
40232,新华社,新华社照片，外代，2017年4月25日\n（外代二线）繁忙的新加坡港\n4月24日，一艘货船...


In [20]:
#处理标签，新华社为1，其他为0
labels = [1 if i =='新华社' else 0 for i in data['source']]
#去掉特殊符号，只保留中英文
texts = [re.sub('[^A-Za-z\u4e00-\u9fa5]','', sent) for sent in data['content']] 
#分词，因为数据集比较大，需要几分钟
cut_texts = [' '.join(list(jieba.cut(sent))) for sent in tqdm_notebook(texts)]

Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\Geoffrey\AppData\Local\Temp\jieba.cache
Loading model cost 1.013 seconds.
Prefix dict has been built succesfully.


In [26]:
#划分数据集，后面5000条为数据测试集，其他为训练集
train_data = cut_texts[:-5000]
train_label = labels[:-5000]
test_data = cut_texts[-5000:]
test_label = labels[-5000:]

In [27]:
#用sklearn里的TfidfVectorizer工具对文本表示
tfidf_model = TfidfVectorizer(max_df = 0.6, max_features =5000).fit(train_data) ## 词表太大，限制特征数为5000
train_data = tfidf_model.transform(train_data)
test_data = tfidf_model.transform(test_data)

In [28]:
#选择xgboost作为分类器进行训练
model = XGBClassifier(n_jobs=4)
model.fit(train_data,train_label)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=4,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [34]:
#查看测试结果
pred_label = model.predict(test_data)
print(classification_report(test_label,pred_label,digits=4))

              precision    recall  f1-score   support

           0     0.9526    0.9448    0.9487       489
           1     0.9940    0.9949    0.9945      4511

    accuracy                         0.9900      5000
   macro avg     0.9733    0.9698    0.9716      5000
weighted avg     0.9900    0.9900    0.9900      5000



In [ ]:
#在二分类里，macro avg就是我们常说的精确率，召回率，F2分数。support 就是样本数量

### Congratulations! You have completed all assignments in this week. The question below is optional. If you still have time, why don't try it out.

## Option:

#### Try differnt machine learning algorithms with different combinations of parameters in the practical part, and compare their performances (Better use some visualization techiniques).

## SVM

In [35]:
from sklearn.svm import LinearSVC

svc_model = LinearSVC()
svc_model.fit(train_data,train_label)

LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [38]:
pred_label = svc_model.predict(test_data)
print(classification_report(test_label,pred_label,digits=4))

              precision    recall  f1-score   support

           0     0.9381    0.8671    0.9012       489
           1     0.9857    0.9938    0.9897      4511

    accuracy                         0.9814      5000
   macro avg     0.9619    0.9304    0.9455      5000
weighted avg     0.9810    0.9814    0.9811      5000



## Random Forest

In [40]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=100,n_jobs=4)# estimator：数据对象   n_jobs：同时工作的cpu个数（-1代表全部）
rf_model.fit(train_data,train_label)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=4,
                       oob_score=False, random_state=None, verbose=0,
                       warm_start=False)

In [43]:
pred_label = rf_model.predict(test_data)
print(classification_report(test_label,pred_label,digits=4))

              precision    recall  f1-score   support

           0     0.9749    0.7955    0.8761       489
           1     0.9783    0.9978    0.9879      4511

    accuracy                         0.9780      5000
   macro avg     0.9766    0.8966    0.9320      5000
weighted avg     0.9779    0.9780    0.9770      5000

